In [3]:
import numpy as np
from ggce import Model
from ggce import System
from ggce import DenseSolver
import matplotlib.pyplot as plt
import os

In [ ]:
Omegas = np.linspace(3,4,100)
couplings = np.linspace(0,0.1,100)
k=0.0
w = np.linspace(-4,4,400)
full_data = {}
full_data["omega_grid"] = w
full_data["k/pi"] = k
full_data["model_type"] = "Holstein"
full_data["M"] = 2
full_data["N"] = 4
full_data["Omega"] = Omegas
full_data["dimensionless_coupling"]=couplings

In [ ]:
if not os.path.exists(os.path.join(os.getcwd(),'2x2gridmultiparameter_spectralfunctions')):
    os.mkdir(os.path.join(os.getcwd(),'2x2gridmultiparameter_spectralfunctions'))

In [ ]:
%%capture
import warnings
warnings.filterwarnings("ignore")
spectras = np.zeros((len(couplings),len(Omegas),len(w)))
for coup_idx,coup in enumerate(couplings):
    omeg_spectras_for_each_coupling = np.zeros((len(Omegas),len(w)))
    for omeg_idx,phon_freq in enumerate(Omegas):
        model = Model.from_parameters(hopping=1.0)
        model.add_(
                    "Holstein",
                    phonon_frequency= phon_freq, #Omega Larger is easier to converge
                    phonon_extent=2,    #Cloud extent
                    phonon_number=4,    #number of phonons
                    dimensionless_coupling_strength=coup     #g    The smaller the easier to converge
                    #coupling_strength=2.0
                    )
        system= System(model)
        solver = DenseSolver(system)
        G = solver.greens_function(k,w,eta=0.05,pbar=True)
        A = -G.imag/np.pi
        spectras[coup_idx,omeg_idx,:] = A
        omeg_spectras_for_each_coupling[omeg_idx,:] = A
    filename = f"Spectras_{len(Omegas)}Omegas_{Omegas[0]}to{Omegas[-1]}_for_coupling_{round(coup,3)}"
    np.save(os.path.join(os.getcwd(),'2x2gridmultiparameter_spectralfunctions',filename),omeg_spectras_for_each_coupling)

full_data["A"] = spectras
np.save("2x2gridmultiparameter_spectralfunctions/All_multiparameter_spectras_100x100x400_Omega3to4_coupling0to_01",spectras)